In [1]:
! pip install cloudstorage



You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
pd.__version__

'1.1.0'

In [3]:
import tensorflow as tf

In [4]:
import keras

Using TensorFlow backend.


In [5]:
import tensorflow.keras as tfk

In [6]:
#!pip install tqdm
#!pip install htop

import pandas as pd
import pickle
import numpy as np
import os
import pickle
from tqdm import tqdm
import json
from joblib import Parallel, delayed
import multiprocessing

from tqdm import tqdm_notebook
import joblib
import sys

import scipy.signal as sp
from scipy.ndimage import median_filter
from scipy.ndimage import uniform_filter
import time
import traceback
from joblib.externals.loky import set_loky_pickler
from joblib import parallel_backend
from joblib import Parallel, delayed
import scipy.stats as sps
from joblib import wrap_non_picklable_objects

import cv2
from PIL import Image

import matplotlib as plt

In [7]:
from sklearn.tree import DecisionTreeClassifier
#clf = DecisionTreeClassifier(random_state=0)
from sklearn.svm import SVC
from sklearn.linear_model import Perceptron
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier

In [8]:
import joblib
from joblib import Parallel, delayed
from joblib.externals.loky import set_loky_pickler
from joblib import parallel_backend
from joblib import Parallel, delayed
from joblib import wrap_non_picklable_objects

import multiprocessing

In [9]:
from keras.models import model_from_json

In [11]:
from sklearn.externals import joblib 

In [12]:
from google.cloud import storage
from io import BytesIO

# Loading PD Data

In [13]:
with open("train_new.pkl", "rb") as input_file:#("/home/jupyter/Adi Part 4/liver-train-val.pkl", "rb") as input_file:case_data_train4#case_data_train1_3.pkl#case_data_train4.pkl
    train=pd.read_pickle(input_file)#pickle.load(input_file)
''' sanse: dataframe with patient information'''
#train=pickle.load('case_data_train1_3.pkl')

In [12]:
train.columns

Index(['caseID', 'name', 'Patient_id', 'Repeated?', 'Site', 'Height', 'Weight',
       'BMI', 'Age', 'Gender', 'History1', 'History2', 'History3',
       'History hbv', 'History hcv', 'History (yes/no)', 'KPA',
       'Fibrosis Grade', 'CAP', 'Steatosis Grade', 'Disease Labelled',
       'Disease', 'Unlabelled Disease', 'Payable', 'irb', 'Ascites',
       'Portal Vein Thrombosis', 'Biopsy', 'Jaundice', 'shadowing_test',
       'Label'],
      dtype='object')

In [14]:
np.shape(train)

(1509, 14)

In [15]:
train.head()

,caseID,name,Disease,History,BMI,FibroscanCAP,FibroscanKPA,Country,Gender,Age,number_of_lesions,lesion_size,Notes,shadowing_test
0,PDG1556036195941,PDG1556036195941,NaN,NaN,22.0,NaN,NaN,Egypt,1.0,65.0,NaN,NaN,NaN,"[[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."
1,PDG1556103181615,PDG1556103181615,NaN,NaN,19.0,NaN,NaN,Egypt,1.0,25.0,NaN,1.0,test,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,PDG1556357075099,PDG1556357075099,[cirrhosis],[hcv],25.0,NaN,NaN,Egypt,1.0,67.0,NaN,4.0,NaN,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,PDG1556384740608,PDG1556384740608,[cirrhosis],[hcv],25.0,NaN,NaN,Egypt,NaN,25.0,NaN,25.0,pending upload,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,PDG1556440560110,PDG1556440560110,[cirrhosis],[hcv],26.0,NaN,NaN,Egypt,NaN,78.0,NaN,5.0,NaN,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."


In [14]:
import copy
''' sanse: Just checking out on tumorous diseases '''
PD=copy.deepcopy(train)
showy=train[((PD['Disease'].str.contains('Nodule', case = False)) | (PD['Disease'].str.contains('tumor', case = False)) |(PD['Disease'].str.contains('lesion', case = False)) |(PD['Disease'].str.contains('hcc', case = False)))]

In [15]:
PD['KPA']= [float(str(val).replace(' ','')) for val in PD['KPA'].values]
PD['CAP']= [float(str(val).replace(' ','')) for val in PD['CAP'].values]
''' sanse: Debug empty KPA and CAP values'''

In [16]:
healthy=PD[(PD['KPA']<7) & ~((PD['Disease'].str.contains('Other', case = False))|(PD['Disease'].str.contains('hcc', case = False)))]#train[train['KPA']<7]
#PD=train[(train['Label']==6) | (train['Label']==5)]#changed for now
Fibtwo=PD[(PD['KPA']>=7)&(PD['KPA']<9) & ~((PD['Disease'].str.contains('Other', case = False))|(PD['Disease'].str.contains('hcc', case = False)))]

Fibthree=PD[(PD['KPA']>=9)&(PD['KPA']<17) & ~((PD['Disease'].str.contains('Other', case = False))  |(PD['Disease'].str.contains('hcc', case = False)))]

#PD=train[(train['Label']==6) | (train['Label']==5)]#
Cirr=PD[(PD['KPA']>=17) &(PD['KPA']<80)& ~((PD['Disease'].str.contains('Other', case = False))  |(PD['Disease'].str.contains('hcc', case = False)))]

'''sanse: OnX thresholds. Skip unclear clinical cases (cancers, nodules, etc.)'''

In [17]:
import copy

In [18]:
healthynames2 = np.array(healthy['caseID'].tolist())
Fibtwonames2=np.array(Fibtwo['caseID'].tolist())
Fibthreenames2=np.array(Fibthree['caseID'].tolist())

In [19]:
import copy
Fibthreenames2=np.array(Fibthree['caseID'].tolist())
Cirrnames2=np.array(Cirr['caseID'].tolist())
c3=copy.deepcopy(Cirrnames2)

In [20]:
Cirrnames2=np.array(Cirr['caseID'].tolist())
c3=copy.deepcopy(Cirrnames2)

In [23]:
np.shape(healthynames2)

(530,)

In [31]:
np.shape(c3)

(18,)

In [37]:
f33=Fibthreenames2

In [27]:
print(np.shape(h3))
print(np.shape(f23))
print(np.shape(f33))
print(np.shape(c3))

(233,)
(40,)
(65,)
(55,)


In [25]:
c3k=np.arange(0,np.shape(c3)[0])
f3k=np.arange(0,np.shape(f33)[0])
f2k=np.arange(0,np.shape(f23)[0])
h3k=np.arange(0,np.shape(h3)[0])

In [30]:
print(np.shape(h3k))
print(np.shape(f2k))
print(np.shape(f3k))
print(np.shape(c3k))

(5,)
(5, 8)
(5, 13)
(5, 11)


In [25]:
print(np.shape(h3))
print(np.shape(f23))
print(np.shape(f33))
print(np.shape(c3))

(233,)
(40,)
(65,)
(55,)


In [13]:
c3k=np.arange(0,np.shape(c3)[0])
'''sanse: indexing the patients'''

In [21]:
h3=healthynames2
f23=Fibtwonames2
f33=Fibthreenames2

In [22]:
h3kpa=healthy['KPA'].tolist()
f2kpa=Fibtwo['KPA'].tolist()
f3kpa=Fibthree['KPA'].tolist()
c3kpa=Cirr['KPA'].tolist()

In [23]:
h3cap=healthy['CAP'].tolist()
f2cap=Fibtwo['CAP'].tolist()
f3cap=Fibthree['CAP'].tolist()
c3cap=Cirr['CAP'].tolist()

In [24]:
f3kpa=Fibthree['KPA'].tolist()
c3kpa=Cirr['KPA'].tolist()
f3cap=Fibthree['CAP'].tolist()
c3cap=Cirr['CAP'].tolist()

In [26]:
# If the data has already been split, load the split
c3=np.load('/home/jupyter/AdisStuff/kfold_new38/c3.npy' )#
f33=np.load('/home/jupyter/AdisStuff/kfold_new38/f33.npy')#)
f23=np.load('/home/jupyter/AdisStuff/kfold_new38/f23.npy')#)
h3=np.load('/home/jupyter/AdisStuff/kfold_new38/h3.npy' )#

c3k=np.load('/home/jupyter/AdisStuff/kfold_new38/c3k.npy',allow_pickle=True)
f3k=np.load('/home/jupyter/AdisStuff/kfold_new38/f3k.npy',allow_pickle=True)
f2k=np.load('/home/jupyter/AdisStuff/kfold_new38/f2k.npy',allow_pickle=True)
h3k=np.load('/home/jupyter/AdisStuff/kfold_new38/h3k.npy',allow_pickle=True)
'''sanse: Reload k-fold split (given split of patients). 5-fold crossvalidations'''
'''sanse: c3, f33, f23, h3: names, c3k, f3k, f2k, h3k: indices for splitting'''

In [28]:
import random

In [29]:
# input variables are described below
# video : a single entry of RF_list (from patient_loader) - RF_list is the RF (signal) data for all slices of all frames of all videos
# threshold : the LOWER LIMIT of what is considered a normal difference between consecutive signal point, in non-shadowing data. A greater value means more leniency in assigning deadspots (potential shadow)
# deadlength : the region size when searching for deadspots within a signal. 350 seems reasonable as each slice gives ~2800 data points
# shift : the shift applied to regions when scanning through a slice (signal) in search of deadspots. A greater shift means less overlap between consecutive regions. Some overlap is needed as a safety net
# deadavg : used on slices, when looking at regions of size deadlength, with values of 0 (dead) or 1 (alive) for each data point. An average along the region of below "deadavg" implies a deadspot
# shadowing_threshold: threshold for % area of frame that is shadowing

def video_shadowing_quicker( video, threshold, deadlength, shift, deadavg, shadowing_threshold ):
    '''sanse: % of each frame that has shadow. it is validated. its based on data science (intern Anton / Breah) '''

    signal_threshold = 100
    
    videoshape = np.shape(video)
    numFrames = videoshape[0]
    slices = videoshape[2]
    numPoints = videoshape[1]
    videoDeadFrames = np.zeros(numFrames)
    
    #number of regions used for each signal
    sizeup = numPoints - 1
    numRegions = (int)(1 + (int) ( (sizeup-deadlength)/shift ) )
    
    deadspots = np.zeros((numFrames , slices , numRegions ))
    deadspots_disp = np.zeros(( numFrames , numPoints, slices ))
    
    #first, find difference between each pair of consecutive points
    diff = np.diff( video , axis = 1 )
    
    #organize each difference as a 1 or 0, wrt some threshold difference we would expect for non-shadowing data
    
    thresh = 1.0 * ( np.abs( diff ) > threshold)
    
    startspot = np.array( [ (shift*ii) for ii in range(0, numRegions) ] )
    endspot = np.add( startspot , deadlength )
    endspot[numRegions-1] = sizeup
    
    avgLife = np.zeros( (numFrames , numRegions , slices) )
    
    for kk in range (0, numFrames):
        avgLife[ kk , :, : ] = np.array( [ [ (np.mean( thresh[ kk , startspot[ii] : endspot[ii]  , jj ] )) for jj in range (0, slices) ] for ii in range( 0 , numRegions ) ]  )
    
    deadspot = np.array(  [ [ [ (( avgLife[kk, ii, jj] < deadavg ) and ( np.mean(np.abs(video[kk, startspot[ii]:endspot[ii]+1 , jj])) < signal_threshold )) for ii in range( 0, numRegions) ] for jj in range(0, slices)]  for kk in range( 0, numFrames) ] )
    deadspot = np.multiply( deadspot , 1 )
    # make an array that simplifies the display of the deadspaces -> if the regions is dead, fill it with 1s
    deadspots_disp = np.zeros( (numFrames, numPoints , slices) )
    
    #fill in deadspots_disp
    for kk in range (0, numFrames):
        for jj in range( 0 , slices ):
            for ii in range( 0 , numRegions ):
                if (deadspot[kk, jj , ii] == 1):
                    deadspots_disp[kk, startspot[ii] :endspot[ii] , jj ] = 1
    
    total = slices*numPoints
    
    dead = np.zeros(numFrames)
    shadowing_fraction = np.zeros(numFrames)
    for kk in range (0, numFrames):
        dead[kk] = np.sum(deadspots_disp[kk,:,:])
        shadowing_fraction[kk] = dead[kk]/total
    
    # ceil to 2 decimal places
    shadowing_fraction = 0.01 * (np.ceil( np.multiply(100.0,shadowing_fraction) ))
    #frame_check = [ 1.0*(shadowing_fraction[ii] < shadowing_threshold) for ii in range(0, numFrames) ]## what I need
    '''
    video_check = np.sum(frame_check) / numFrames
    
    # now search for regions of the video ( size aka # of frames = 0.3 * total # of frames )
    # where 8-9/10 frames are good -> ie. the avg of framecheck over these frames needs to be 0.8-0.9 or greater
    chunkFrames = (int)(np.ceil( 0.3*numFrames ))
    numChunks = 1 + numFrames - chunkFrames
    chunkFound = 0
    counter = (int)(0)
    avgQual = 0
    avgShad = 1  
    
    # v2.0 - Looks for a region with shadowing_threshold or better shadowing on average
    while ( (avgShad >= shadowing_threshold) and (counter < numChunks) ):
        avgShad = np.mean( shadowing_fraction[counter:counter+chunkFrames] )
        counter += 1
    if ( avgShad <= shadowing_threshold ):
        chunkFound = 1
    '''
    return shadowing_fraction#actually is frame_check#video_check, chunkFound

In [30]:
# input variables are described below
# video : a single entry of RF_list (from patient_loader) - RF_list is the RF (signal) data for all slices of all frames of all videos
# threshold : the LOWER LIMIT of what is considered a normal difference between consecutive signal point, in non-shadowing data. A greater value means more leniency in assigning deadspots (potential shadow)
# deadlength : the region size when searching for deadspots within a signal. 350 seems reasonable as each slice gives ~2800 data points
# shift : the shift applied to regions when scanning through a slice (signal) in search of deadspots. A greater shift means less overlap between consecutive regions. Some overlap is needed as a safety net
# deadavg : used on slices, when looking at regions of size deadlength, with values of 0 (dead) or 1 (alive) for each data point. An average along the region of below "deadavg" implies a deadspot
# shadowing_threshold: threshold for % area of frame that is shadowing

def video_shadowing_quicker2( video, threshold, deadlength, shift, deadavg, shadowing_threshold ):
    '''sanse: give a threshold and returns the binary (quality check on frames aspect)'''

    signal_threshold = 100
    
    videoshape = np.shape(video)
    numFrames = videoshape[0]
    slices = videoshape[2]
    numPoints = videoshape[1]
    videoDeadFrames = np.zeros(numFrames)
    
    #number of regions used for each signal
    sizeup = numPoints - 1
    numRegions = (int)(1 + (int) ( (sizeup-deadlength)/shift ) )
    
    deadspots = np.zeros((numFrames , slices , numRegions ))
    deadspots_disp = np.zeros(( numFrames , numPoints, slices ))
    
    #first, find difference between each pair of consecutive points
    diff = np.diff( video , axis = 1 )
    
    #organize each difference as a 1 or 0, wrt some threshold difference we would expect for non-shadowing data
    
    thresh = 1.0 * ( np.abs( diff ) > threshold)
    
    startspot = np.array( [ (shift*ii) for ii in range(0, numRegions) ] )
    endspot = np.add( startspot , deadlength )
    endspot[numRegions-1] = sizeup
    
    avgLife = np.zeros( (numFrames , numRegions , slices) )
    
    for kk in range (0, numFrames):
        avgLife[ kk , :, : ] = np.array( [ [ (np.mean( thresh[ kk , startspot[ii] : endspot[ii]  , jj ] )) for jj in range (0, slices) ] for ii in range( 0 , numRegions ) ]  )
    
    deadspot = np.array(  [ [ [ (( avgLife[kk, ii, jj] < deadavg ) and ( np.mean(np.abs(video[kk, startspot[ii]:endspot[ii]+1 , jj])) < signal_threshold )) for ii in range( 0, numRegions) ] for jj in range(0, slices)]  for kk in range( 0, numFrames) ] )
    deadspot = np.multiply( deadspot , 1 )
    # make an array that simplifies the display of the deadspaces -> if the regions is dead, fill it with 1s
    deadspots_disp = np.zeros( (numFrames, numPoints , slices) )
    
    #fill in deadspots_disp
    for kk in range (0, numFrames):
        for jj in range( 0 , slices ):
            for ii in range( 0 , numRegions ):
                if (deadspot[kk, jj , ii] == 1):
                    deadspots_disp[kk, startspot[ii] :endspot[ii] , jj ] = 1
    
    total = slices*numPoints
    
    dead = np.zeros(numFrames)
    shadowing_fraction = np.zeros(numFrames)
    for kk in range (0, numFrames):
        dead[kk] = np.sum(deadspots_disp[kk,:,:])
        shadowing_fraction[kk] = dead[kk]/total
    
    # ceil to 2 decimal places
    shadowing_fraction = 0.01 * (np.ceil( np.multiply(100.0,shadowing_fraction) ))
    frame_check = [ 1.0*(shadowing_fraction[ii] < shadowing_threshold) for ii in range(0, numFrames) ]## what I need
    '''
    video_check = np.sum(frame_check) / numFrames
    
    # now search for regions of the video ( size aka # of frames = 0.3 * total # of frames )
    # where 8-9/10 frames are good -> ie. the avg of framecheck over these frames needs to be 0.8-0.9 or greater
    chunkFrames = (int)(np.ceil( 0.3*numFrames ))
    numChunks = 1 + numFrames - chunkFrames
    chunkFound = 0
    counter = (int)(0)
    avgQual = 0
    avgShad = 1  
    
    # v2.0 - Looks for a region with shadowing_threshold or better shadowing on average
    while ( (avgShad >= shadowing_threshold) and (counter < numChunks) ):
        avgShad = np.mean( shadowing_fraction[counter:counter+chunkFrames] )
        counter += 1
    if ( avgShad <= shadowing_threshold ):
        chunkFound = 1
    '''
    return frame_check#video_check, chunkFound

In [31]:
def scanconvert(Iin=None, info=None):#Apitch, Lpitch, Radius, PixelsPerMM)
    '''sanse: same as in MaskingCleaned'''
    import numpy as np
    if (info["Radius"] == 0):    # A linear array probe
        print("Linear Array")
        AxialExtent = info["Apitch"] * np.size(Iin, 0)
        LateralExtent = info["Lpitch"] * np.size(Iin, 1)
        AxialSize = AxialExtent * 1000 * info["PixelsPerMM"]
        LateralSize = LateralExtent * 1000 * info["PixelsPerMM"]
        Iout = np.resize(Iin, (LateralSize, AxialSize))    # cubic interpolation
    else:    # A curved array probe    
        t = ((np.arange(1,Iin.shape[1],dtype=int)) - Iin.shape[1]/ 2) * info["Lpitch"] / info["Radius"]
        r = info["Radius"] + (np.arange(1,Iin.shape[0])) * info["Apitch"]
        [t, r] = np.meshgrid(t, r)
        x = np.multiply(r, np.cos(t))
        y = np.multiply(r, np.sin(t))
    
        divides=1e-3 / info["PixelsPerMM"]
        xMin=np.min(x)
        xMax=np.max(x)
        #print("X Min : %s , X Max %s" % (xMin,xMax))
        #print("Divides : %s " % divides)
        xreg =  np.arange(np.min(x) , np.max(x) ,divides)
        yreg =  np.arange(np.min(y) , np.max(y) ,divides)
        [yreg, xreg] = np.meshgrid(yreg, xreg) 
        Iout = np.zeros(xreg.shape)
        xCntr = np.arange(0,xreg.shape[1])
        Cntry = np.arange(0,yreg.shape[0])
        theta = np.array([cart4pol(xreg[yCntr, xCntr], yreg[yCntr, xCntr]) for yCntr in Cntry])
        rho = np.array([cart2pol(xreg[yCntr, xCntr], yreg[yCntr, xCntr]) for yCntr in Cntry])
        #for xCntr in tqdm(np.arange(0,xreg.shape[1])):
            #for yCntr in np.arange(0,yreg.shape[0]):
                #[theta, rho] = cart2pol(xreg[yCntr, xCntr], yreg[yCntr, xCntr])
                #print("theta : %s , rho %s" % (theta,rho))
        indt = np.floor(theta / (info["Lpitch"] / info["Radius"]) + Iin.shape[1] / 2) 
        indr = np.floor((rho - info["Radius"]) / info["Apitch"]) 

        [j,i]=((indt>=0)&(indt <= t.shape[1])& (indr>= 0) &(indr <= r.shape[0])).nonzero()
        #print(np.size(j))
        j=j.astype(int)
        i=i.astype(int)
        Iout[j, i] = Iin[indr[j,i].astype(int), indt[j,i].astype(int)]
        #for i in xCntr:
            
         #   for j in Cntry:
                
          #      if indt[j,i] >= 0 and (indt[j,i] <= t.shape[1]and (indr[j,i] >= 0 and indr[j,i] <= r.shape[0])):
                    
           #         Iout[int(Cntry[j]), int(xCntr[i])] = Iin[int(indr[j,i]), int(indt[j,i])]
    #print(np.shape(Iout))
    return Iout

def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    return rho
def cart4pol(x, y):
    phi = np.arctan2(y, x)
    return phi


def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)


In [32]:
def rf2bmode(rf):
    '''sanse: same as DSP_Part1'''
    import numpy as np
    from scipy.signal import hilbert,decimate,resample


    # decimation factor use to create Env from RF
    decimationFactor = 1

    # make a compression table
    alpha = 0.55# sqrt compression, change for a different compression table
    denom = np.exp(alpha * np.log(65535)) / 255.0
    CompressionTable = np.exp(alpha * np.log(np.arange(0,65536))) / denom 
    # CompressionTable = np.exp(alpha * np.log(65535)) / denom
    
    # calculate envelope and log compress
    Env = 1 + np.fix(np.abs(hilbert(rf, axis= 0)))
    Env = CompressionTable[np.array(Env, dtype=int)]

    EnvDec =resample(Env ,int(len(Env[ : , 0])/decimationFactor))
    
    EnvDec[(EnvDec>255).nonzero()]=255
#EnvDec[(EnvDec>255).nonzero()]=255
    return EnvDec

In [39]:
'''
"BackScatter Coefficient Rectangular Window"))-0
 "ESD Rectangular Window"))-1
     "EAC Rectangular Window"))-2
 "Spectral Slope Rectangular Window"))-3
    "Spectral Intercept Rectangular Window"))-4
  "MidBand Fit Rectangular Window"))-5
 "Attenuation Coefficient Hamming Window"))-6
"BackScatter Coefficient Hamming Window"))-7
 "ESD Hamming Window"))-8
 "EAC Hamming Window"))-9
 "Spectral Slope Hamming Window"))-10
"Spectral Intercept Hamming Window"))-11
 "MidBand Fit Hamming Window"))-12
      "Nakagami Spread"))-13
 "Nakagami Shape"-14
'''


'\n"BackScatter Coefficient Rectangular Window"))-0\n "ESD Rectangular Window"))-1\n     "EAC Rectangular Window"))-2\n "Spectral Slope Rectangular Window"))-3\n    "Spectral Intercept Rectangular Window"))-4\n  "MidBand Fit Rectangular Window"))-5\n "Attenuation Coefficient Hamming Window"))-6\n"BackScatter Coefficient Hamming Window"))-7\n "ESD Hamming Window"))-8\n "EAC Hamming Window"))-9\n "Spectral Slope Hamming Window"))-10\n"Spectral Intercept Hamming Window"))-11\n "MidBand Fit Hamming Window"))-12\n      "Nakagami Spread"))-13\n "Nakagami Shape"-14\n'

# Code to access the buckets

In [39]:
bucket_pull

<Bucket: train-new>

In [37]:
import cloudstorage as gcs
from google.cloud import storage
storage_client = storage.Client()

bucket_pull_name ="train-new"#"liver-data-train-4-n"#"liver-data-48x61-alldata-test-1"#"liver-data-train-val" "liver-data-48x61-alldata-train-n"#

bucket_pull = storage_client.get_bucket(bucket_pull_name)

In [38]:
def get_info_video(data_type):
    length = len(data_type) + 3
    # get all patient IDS - use the rf VIDEO folder for simplicity
    # get the blob iterator objects
    blob = bucket_pull.list_blobs()
    # get relevant file names from blob objects
    fileList1 = [file.name for file in blob if ( file.name.endswith((data_type+'.npy')) and file.name.startswith((data_type+'_v/')) ) ]
    fileList1 = np.asarray(fileList1)
    size = np.size(fileList1)
    ID_list1 = np.copy(fileList1)
    for ii in range(0, size):
        ID_list1[ii] = fileList1[ii][length:length+16]
    # avoid duplicates
    ID_list1 = np.unique(ID_list1)
    # get all video names
    size = np.shape(fileList1)[0]
    fileList = np.copy(fileList1)
    caseID = np.copy(fileList)
    videID = np.copy(fileList)
    bothID = np.copy(fileList)
    for ii in range(0, size):
        fileList[ii] = fileList1[ii][length:]
        split = fileList[ii].split('_')
        caseID[ii] = split[0]
        videID[ii] = split[1]
        bothID[ii] = split[0] + '_' + split[1]
    caseID = np.unique(caseID)
    videID = np.unique(videID)
    bothID2 = np.unique(bothID)
    return caseID, videID, bothID2

In [39]:
caseID, videID, bothID2 = get_info_video('qus')

In [41]:
# get videos and frame numbers for each case
info_list = []

for ii in range(0, np.shape(caseID)[0]):
    # set up info list
    info_list.append( [] )
    
    # get ID
    info_list[ii].append(caseID[ii])
    
    # get video list
    vidlist = [ jj for jj in bothID2 if caseID[ii] in jj ]
    vidlist =  [ jj[17:] for jj in vidlist ]
    info_list[ii].append(vidlist)

In [42]:
info_dict = {}
case_dict = {}
for ii in range(0, len(info_list)):
    
    case_dict = {'videos': info_list[ii][1] }
    
    info_dict[info_list[ii][0]] = case_dict

In [43]:
# load video directly from _v/ folder
def load_qus_video_direct( ID , vid ):   
    filename = 'qus/' + ID + '_' + vid + '_qus.npy' 
    temp = storage.blob.Blob(filename,bucket_pull)
    content = temp.download_as_string()
    video = np.load(BytesIO(content))
    return video

# load video directly from _v/ folder
def load_rf_video_direct( ID , vid ):   
    filename = 'rf/' + ID + '_' + vid + '_rf.npy' 
    temp = storage.blob.Blob(filename,bucket_pull)
    content = temp.download_as_string()
    video = np.load(BytesIO(content))
    return video

# load video directly from _v/ folder
def load_quant_video_direct( ID , vid ):   
    filename = 'quant/' + ID + '_' + vid + '_quant.npy' 
    temp = storage.blob.Blob(filename,bucket_pull)
    content = temp.download_as_string()
    video = np.load(BytesIO(content))
    return video

In [44]:
# load video directly from _v/ folder
def load_qus_video_direct2( ID , vid ):   
    filename = 'qus_v/' + ID + '_' + vid + '_qus.npy' 
    temp = storage.blob.Blob(filename,bucket_pull)
    content = temp.download_as_string()
    video = np.load(BytesIO(content))
    return video

# load video directly from _v/ folder
def load_rf_video_direct2( ID , vid ):   
    filename = 'rf_v/' + ID + '_' + vid + '_rf.npy' 
    temp = storage.blob.Blob(filename,bucket_pull)#bucket_pull_2
    content = temp.download_as_string()
    video = np.load(BytesIO(content))
    return video

# load video directly from _v/ folder
def load_quant_video_direct2( ID , vid ):   
    filename = 'quant_v/' + ID + '_' + vid + '_quant.npy' 
    temp = storage.blob.Blob(filename,bucket_pull)
    content = temp.download_as_string()
    video = np.load(BytesIO(content))
    return video

In [45]:
# bucket_pull_name_2 = "liver-data-48x61-alldata-train"#""liver-data-train-4"#"liver-data-48x61-alldata-test-1"#"liver-data-train-val" "liver-data-48x61-alldata-train"#"

# bucket_pull_2 = storage_client.get_bucket(bucket_pull_name_2)

In [46]:
# bucket_pull_name_3 = "liver-data-48x61-alldata-train-n-3"#"liver-data-48x61-alldata-train-n"#""liver-data-train-4-n"#"liver-data-48x61-alldata-test-1-n"#"liver-data-train-val" "liver-data-48x61-alldata-train-n"#"

# bucket_pull_3 = storage_client.get_bucket(bucket_pull_name_3)

In [ ]:
# 'newmodel'+files[m][2:-6]+'msk.npy') ), 'w') , predicted_mask_good )
# 'prctmodel'+files[m][2:-6]+'msk.npy') ), 'w') , predicted_mask_bad )

In [47]:
def load_fullmask_video_direct2( ID , vid ):   
    #filename = 'newmodel_v/' + ID + '_' + vid + '_msk.npy' #switch new for old newmodel_v
    filename = 'bigmask_v/' + ID + '_' + vid + '_msk.npy'
    temp = storage.blob.Blob(filename,bucket_pull)#bucket_pull_3
    content = temp.download_as_string()
    video = np.load(BytesIO(content))
    return video
def load_prcntmask_video_direct2( ID , vid ):   
    #filename = 'prctmodel_v/' + ID + '_' + vid + '_msk.npy' 
    filename = 'origmsk_v/' + ID + '_' + vid + '_msk.npy' #
    temp = storage.blob.Blob(filename,bucket_pull)#_3
    content = temp.download_as_string()
    video = np.load(BytesIO(content))
    return video
def load_ps_video_direct2( ID , vid ):   
    #filename = 'prctmodel_v/' + ID + '_' + vid + '_msk.npy' 
    filename = 'ps_v/' + ID + '_' + vid + '_ps.npy' #
    temp = storage.blob.Blob(filename,bucket_pull)
    content = temp.download_as_string()
    video = np.load(BytesIO(content))
    return video

# If the data hasn't been split into fold it happens here

In [48]:
def K_fold_split(a3,k):
    '''sanse: k-fold split. It is stratified in advance, and the split is then done for each disease individually.'''
    cor=np.arange(0,np.shape(a3)[0])
    cora=copy.deepcopy(cor)
    random.shuffle(cora)
    f45=np.array_split(cora,k)
    return f45


In [49]:
c3k=K_fold_split(c3,5)
f3k=K_fold_split(f33,5)
f2k=K_fold_split(f23,5)
h3k=K_fold_split(h3,5)

In [50]:
print(h3k)
print(f2k)
print(f3k)
print(c3k)

[array([227,  51, 171,  32,  28,  88,  11,  35, 166, 164, 107, 201,  39,
       129, 150,  95, 218, 169,  93,  89,  25, 183,  72,  14, 118, 136,
        20,  85, 211,  65,  61,  18, 231, 190, 225,   1,  94, 187, 147,
       148, 216, 195,  24, 139, 210,  90, 145]), array([109,   9,  41, 101, 175, 191, 149, 165,  31, 144,  87, 137, 162,
       161, 167,  83, 177, 185,  80, 212,  19, 113,  38, 214,  30,  76,
        53, 194,  75, 117,   0,  27,  56,  84, 172,  49, 178,  55, 116,
       138,  99, 134,  96,  68, 203,  37, 141]), array([ 26,  74, 170,  69, 102,  81, 143, 221, 160, 112,  36, 209,  59,
       142, 224, 104, 124,  33,  97, 205,  67, 200, 151,  16,  40,  60,
        22,  44, 222, 146,  12, 131,  86, 230, 188,  91,  73, 121, 173,
       135, 168, 229,  45, 213, 217,  29, 197]), array([228,  52, 123, 202, 226, 220,  34,  46, 127,   8,  21,   2, 155,
       179, 100, 184, 199,  48, 176, 106,  57,  42,   6, 163, 152, 182,
        17,  54,  58,  79, 157, 154,  82, 130,  23, 105, 207

In [49]:
np.shape(f33)

(120,)

In [37]:
print(np.shape(h3k[1]))
print(np.shape(f2k[1]))
print(np.shape(f3k[1]))
print(np.shape(c3k[1]))

(106,)
(19,)
(24,)
(15,)


# This section is for a particular version of the GLCM, it isn't used now, but I kept it for reference.

In [ ]:
##important original
''' GLCM requires discretized data, we pull 8 random patients for defining bins. Not anymore actual code. Now every patient has their own bins'''

In [38]:
qusfixingnames=np.hstack((h3[h3k[0][20]],h3[h3k[4][100]] ,f23[f2k[1][2]],f23[f2k[3][10]]  ,f33[f3k[1][3]],f33[f3k[2][20]]  ,c3[c3k[0][7]],c3[c3k[1][14]]          ))

In [ ]:
##testing here

In [37]:
qusfixingnames=np.hstack((h3[h3k[0][20]],h3[h3k[4][60]] ,f23[f2k[1][2]],f23[f2k[3][8]]  ,f33[f3k[1][3]],f33[f3k[2][10]]  ,c3[c3k[0][2]],c3[c3k[1][1]]          ))

In [36]:
np.shape(h3k[1])

(63,)

In [54]:
np.shape(qusfixingnames)

(8,)

In [40]:

def qusforquant(patient_list, patient_indexes):
    qusfout=[]
    
    for m in tqdm_notebook(range(0,np.shape(patient_indexes)[0])): 
        #DF = Load_QUS_Loc(c3[cr[m]:cr[m]+1])
        #try:
        #    DF['Case']
        #    print(1)
        #except:
        #    continue
        '''
        try:
            QUS_list, QUANT_list, RF_list = Dat_nump_loader(patient_list[patient_indexes[m]])
        except:
            try:
                QUS_list, QUANT_list, RF_list = Dat_nump_loaderval(patient_list[patient_indexes[m]])
            except:
                print(patient_list[patient_indexes[m]])
        '''
        num_cores = multiprocessing.cpu_count()
        
        try:
            with parallel_backend('multiprocessing'):
                QUS_list = Parallel(n_jobs=num_cores,mmap_mode='r+')(delayed(load_qus_video_direct)(patient_list[patient_indexes[m]],info_dict[patient_list[patient_indexes[m]]]['videos'][ii]) for ii in range(0,len(info_dict[patient_list[patient_indexes[m]]]['videos'])) )
            #print('QUS_list shape', np.shape(QUS_list)[0])
            
            with parallel_backend('multiprocessing'):
                RF_list = Parallel(n_jobs=num_cores,mmap_mode='r+')(delayed(load_rf_video_direct)(patient_list[patient_indexes[m]],info_dict[patient_list[patient_indexes[m]]]['videos'][ii]) for ii in range(0,len(info_dict[patient_list[patient_indexes[m]]]['videos'])) )
            #print('RF_list shape', np.shape(RF_list)[0])
           
            #with parallel_backend('multiprocessing'):
            #    QUANT_list = Parallel(n_jobs=num_cores,mmap_mode='r+')(delayed(load_quant_video_direct)(patient_list[patient_indexes[m]],info_dict[patient_list[patient_indexes[m]]]['videos'][ii]) for ii in range(0,len(info_dict[patient_list[patient_indexes[m]]]['videos'])) )
        except:
        
            with parallel_backend('multiprocessing'):
                QUS_list = Parallel(n_jobs=num_cores,mmap_mode='r+')(delayed(load_qus_video_direct2)(patient_list[patient_indexes[m]],info_dict[patient_list[patient_indexes[m]]]['videos'][ii]) for ii in range(0,len(info_dict[patient_list[patient_indexes[m]]]['videos'])) )
            #print('QUS_list shape', np.shape(QUS_list)[0])
            
            with parallel_backend('multiprocessing'):
                RF_list = Parallel(n_jobs=num_cores,mmap_mode='r+')(delayed(load_rf_video_direct2)(patient_list[patient_indexes[m]],info_dict[patient_list[patient_indexes[m]]]['videos'][ii]) for ii in range(0,len(info_dict[patient_list[patient_indexes[m]]]['videos'])) )
            #print('RF_list shape', np.shape(RF_list)[0])
           
            #with parallel_backend('multiprocessing'):
            #    QUANT_list = Parallel(n_jobs=num_cores,mmap_mode='r+')(delayed(load_quant_video_direct2)(patient_list[patient_indexes[m]],info_dict[patient_list[patient_indexes[m]]]['videos'][ii]) for ii in range(0,len(info_dict[patient_list[patient_indexes[m]]]['videos'])) )
        
        
        #print('QUANT_list shape', np.shape(QUANT_list)[0])
        # load the videos
        #with parallel_backend('multiprocessing'):
            #QUS_list = Parallel(n_jobs=num_cores,mmap_mode='r+')(delayed(load_qus_video)(patient_list[patient_indexes[m]],info_dict[patient_list[patient_indexes[m]]]['videos'][ii],info_dict[patient_list[patient_indexes[m]]]['frames'][ii]) for ii in range(0,len(info_dict[patient_list[patient_indexes[m]]]['videos'])) )
        #print('QUANT_list shape', np.shape(QUANT_list)[0])
        #print('QUS_list shape', np.shape(QUS_list)[0])
        
        #QUS_list, QUANT_list, RF_list, File_names = numpy_for_ID_seg(DF,c3[cr[m]])
        #nums=Cirr['2'][Cirr['PDG1556357075099']==c3[cr[m]]].tolist()
        
        
        
                
        for r in range(0,np.shape(RF_list)[0]):#(np.asarray(nums)-1):#range(0,np.shape(QUANT_list)[0]):
            prep=video_shadowing_quicker2((RF_list[r]), 80, 350, 200, 0.08, 0.18) 
            
            if np.shape(RF_list[r])[0]==0:
                datavidtopat[np.shape(datavidtopat)[0]-1]=datavidtopat[np.shape(datavidtopat)[0]-1]-1
                continue
            elif np.max(prep)==0:
                continue
            Rf_list2=[]
            Quant_list2=[]
            Qus_list2=[]
            
            Qus_list2=QUS_list[r][(np.reshape(prep,(-1,)).nonzero()[0]),:,:]
            qusfout.append(Qus_list2)
    return qusfout

In [41]:
yki=qusforquant(qusfixingnames,np.arange(0,8))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [42]:
np.shape(yki)

(51,)

In [43]:
yki2=yki[0]
for m in range(1,51):
    yki2=np.vstack((yki2,yki[m]))

In [44]:
np.shape(yki2)

(1203, 1152, 17)

In [45]:
yki3=yki2[0,:,:]
for m in range(1,1203):
    yki3=np.vstack((yki3,yki2[m,:,:]))

In [46]:
np.shape(yki3)

(1385856, 17)

In [ ]:
(1385856, 17)


In [ ]:
#ORIGINAL

In [58]:
yki=qusforquant(qusfixingnames,np.arange(0,8))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [60]:
np.shape(yki)

(51,)

In [61]:
yki2=yki[0]
for m in range(1,51):
    yki2=np.vstack((yki2,yki[m]))

In [62]:
np.shape(yki2)

(1203, 2304, 17)

In [63]:
yki3=yki2[0,:,:]
for m in range(1,1203):
    yki3=np.vstack((yki3,yki2[m,:,:]))

In [64]:
np.shape(yki3)

(2771712, 17)

In [48]:
def eodev22(values,b):
    ''' sanse: binning version define bin sizes, orders data and puts it in the bins'''
    orders=np.sort(values, axis=0)
    a=np.shape(values)[0]
    incv=np.floor(a/b)
    #ardf=np.zeros(np.shape(values))
    ep=[]
    for i in range(1,b):
        
        ep.append(orders[int(incv*(i)),:])
        
    ep=np.asarray(ep)
    del values
    del orders
    #del ardf
    return (ep)

In [49]:
def badmathflat2(values, ep, b):
''' sanse: applies thresholds'''

    ardf=np.zeros(np.shape(values))    
    ardf[(values[:,:]<ep[0,:]).nonzero()]=0
    ardf[(values[:,:]>=ep[b-2,:]).nonzero()]=b-1

    for i in range(1,b-1):
        ardf[((values[:,:]>=ep[i-1,:]) & (values[:,:]<ep[i,:])).nonzero()]=i
    del values
    #del orders
    #del ardf
    return (ardf)

In [50]:
def eodev(values,b):
    orders=np.sort(values, axis=0)
    a=np.shape(values)[0]
    incv=np.floor(a/b)
    #ardf=np.zeros(np.shape(values))
    ep=[]
    for i in range(0,b):
        if i<b-1:
            ep.append(orders[int(incv*(i+1)),:])
        else:
            ep.append(orders[a-1,:])
    ep=np.asarray(ep)
    del values
    del orders
    #del ardf
    return (ep)

In [51]:
def badmathflat(values, ep, b):
    ardf=np.zeros(np.shape(values))
    #ep=[]
    for x in range(0,17):
        for i in range(0,b):
            if i==0:
                ardf[(values[:,x]<ep[i,x]).nonzero(),x]=i
            elif i==b-1:
                ardf[((values[:,x]>=ep[i-1,x]) ).nonzero(),x]=i
            else:
                ardf[((values[:,x]>=ep[i-1,x]) & (values[:,x]<ep[i,x])).nonzero(),x]=i
    del values
    #del orders
    #del ardf
    return (ardf)

In [69]:
ep = eodev(yki3,64) #swap b=8 with 12 when done testing out 8
print(np.shape(ep))

(64, 17)


# Relevent Code starts Again

In [52]:
# functions for fractal features 
## Minkowski (O.G.) box-counting
'''sanse: not sure if it is a good thing or not, does not look hugely valuable.'''
'''sanse: open-source repository (public), Omar's project.'''
'''sanse: Check. Combine them with better masks.'''
def Minkowski_FD(Z, threshold=0.9):
    # Only for 2d image
    assert(len(Z.shape) == 2)

    # From https://github.com/rougier/numpy-100 (#87)
    
    def boxcount(Z, k):
        S = np.add.reduceat(
            np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0),
                               np.arange(0, Z.shape[1], k), axis=1)

        # We count non-empty (0) and non-full boxes (k*k)
        return len(np.where((S > 0) & (S < k*k))[0])

    # Transform Z into a binary array
    Z = (Z < np.nanmean(Z))#threshold)

    # Minimal dimension of image
    p = min(Z.shape)

    # Greatest power of 2 less than or equal to p
    n = 2**np.floor(np.log(p)/np.log(2))

    # Extract the exponent
    n = int(np.log(n)/np.log(2))

    # Build successive box sizes (from 2**n down to 2**1)
    sizes = 2**np.arange(n, 1, -1)

    # Actual box counting with decreasing size
    counts = []
    for size in sizes:
        counts.append(boxcount(Z, size))

    # Fit the successive log(sizes) with log (counts)
    coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
    return -coeffs[0]


In [53]:
#!pip install hfda
import hfda


In [54]:
from scipy.stats import mode

In [53]:
#!pip install pyradiomics

     |████████████████████████████████| 188 kB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 111 kB 11.4 MB/s eta 0:00:01
     |████████████████████████████████| 547 kB 10.1 MB/s eta 0:00:01
     |████████████████████████████████| 47.4 MB 32 kB/s s eta 0:00:01
  Attempting uninstall: ruamel.yaml
    Found existing installation: ruamel-yaml 0.15.80
    Uninstalling ruamel-yaml-0.15.80:
      Successfully uninstalled ruamel-yaml-0.15.80
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [55]:
import hfda
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm, getTestCase, ngtdm
from radiomics import featureextractor
import SimpleITK as sitk

In [ ]:
#functions for pyradiomics

In [56]:
ep=[]

In [57]:
def eodev22(values,b):
    '''sos: actual version of binning'''
    orders=np.sort(values, axis=0)
    a=np.shape(values)[0]
    incv=np.floor(a/b)
    #ardf=np.zeros(np.shape(values))
    ep=[]
    for i in range(1,b):
        
        ep.append(orders[int(incv*(i)),:])
        
    ep=np.asarray(ep)
    del values
    del orders
    #del ardf
    return (ep)

In [58]:
def badmathflat2(values, ep, b):
    ardf=np.zeros(np.shape(values))    
    ardf[(values[:,:]<ep[0,:]).nonzero()]=0
    ardf[(values[:,:]>=ep[b-2,:]).nonzero()]=b-1

    for i in range(1,b-1):
        ardf[((values[:,:]>=ep[i-1,:]) & (values[:,:]<ep[i,:])).nonzero()]=i
    del values
    #del orders
    #del ardf
    return (ardf)

In [57]:
## For scan-conversion
with open('Iminf_.pkl', 'rb') as handle:
    imgInfo = pickle.load(handle)
print(imgInfo)
imgInfo2=copy.deepcopy(imgInfo)
# Run only ONCE! Adjusts imgInfo to match the downsampling of the QUS heatmaps
imgInfo2["PixelsPerMM"]=2
imgInfo2["Radius"]=imgInfo2["Radius"]/4
print(imgInfo)
'''sanse: two versions, since sth. else is scan-converted'''

{'probeStruct': {'name': 'C7-3/50', 'type': 0, 'transmitoffset': 0, 'vendors': '\n\t\t\t\t\n          \n\t\t\t\t', 'biopsy': '\n\t\t\t\n        \n\t\t\t', 'maxfocusdistance': 300000, 'maxsteerangle': 15000, 'minFocusDistanceDoppler': 80000, 'minlineduration': 120, 'radius': 50420, 'numElements': 128, 'pinOffset': 0, 'pitch': 479, 'support': '\n\t\t\t\n        \n\t\t\t', 'muxWrap': '\n\t\t\t\n        \n\t\t\t', 'elevationLength': 1, 'maxPwPrp': 800, 'invertedElements': 0, 'frequency': {'center': 5000000, 'bandwidth': 4000000}, 'motor': {'FOV': 0, 'homeMethod': 0, 'minTimeBetweenPulses': 0, 'motor_radius': 0, 'steps': 0, 'homeCorrection': 0}}, 'studyMode': 'RF', 'file': 'lev1bad.pkl', 'filepath': '', 'probe': 'clarius', 'system': 'Clarius', 'studyID': 'ev1bad', 'samples': 1831759474, 'lines': 1634036843, 'depthOffset': 0, 'depth': 17692376210, 'width': 230, 'rxFrequency': 20000000, 'samplingFrequency': 20000000, 'txFrequency': 4, 'centerFrequency': 4, 'targetFOV': 0, 'numFocalZones': 1, 

These are for the sinple SP Maps

In [59]:
ph1=np.load('PhantforNak.npy')
'''sms: load up phantom'''

In [55]:
np.shape(ph1)

(2928, 192)

In [60]:
def NakaGamiParam2(c,v):
'''sms: it calculates again NP2 (correct some mistakes in QUS map), and adds some additional stuff (snr). We stack both NPs into the network'''
    from scipy.signal import hilbert
    #c is patient rf data, v is phantom rf data
    r=np.abs(hilbert(c,axis=1))
#     r=hilbert(c,axis=1)
    #print(np.shape(r))
    #p=hilbert(v,axis=1)
    p=np.abs(hilbert(v,axis=1))
    #w=np.nanmean((r)**2)
    #u=((np.nanmean((r)**2))**2)/(np.nanmean((((r)**2)-np.nanmean((r)**2))**2))
    w=np.nanmean((r/p)**2,axis=1)
    #print(np.shape(w))
    u=((np.nanmean((r/p)**2,axis=1))**2)/np.var((r/p)**2,axis=1)#(np.nanmean((((r/p)**2)-np.nanmean((r/p)**2,axis=1).reshape(-1,1,1))**2,axis=1))
    
    #print(np.shape(u))
    w=np.nanmean(w,axis=1)
    u=np.nanmean(u,axis=1)
    srm=np.nanmean(r,axis=1)
    srsd=np.nanstd(r,axis=1)
    srp=srm/srsd
    sr=np.nanmean(srp,axis=1)
    
    #return[np.abs(w),np.abs(u),np.abs(sr)]
    return[w,u,sr]

In [61]:
def setforrad3(trial):
'''sanse: discretize dataset for entropy calculations. Discretization can be improved (check)'''
'''sanse: heuristic method'''
'''sanse: threshold values tuned graphically'''
    ar=np.zeros(np.shape(trial))#initialize a variable for holding new values
    
    #Read up on quantization
    #non linear homebrew method
    (a,b)=(trial<=-3000).nonzero()
    #at=np.floor((trial-np.min(trial))/(-3000-np.min(trial))*25)
    ar[a,b]=np.floor(((trial[a,b]-np.min(trial))/(-3000-np.min(trial)))*25)#at[a,b]

    #at=(np.floor(((trial-(-3000))/(-1000-(-3000)))*25)+25)

    (a,b)=((-3000<trial) & (trial<=-1000)).nonzero()
    ar[a,b]=(np.floor(((trial[a,b]-(-3000))/(-1000-(-3000)))*25)+25)#at[a,b]


    #at=(np.floor(((trial-(-1000))/(-500-(-1000)))*25)+50)
    (a,b)=((-1000<trial) & (trial<=-500)).nonzero()
    ar[a,b]=(np.floor(((trial[a,b]-(-1000))/(-500-(-1000)))*25)+50)#at[a,b]


    #at=(np.floor(((trial-(-500))/(500-(-500)))*100)+75)
    (a,b)=((-500<trial) & (trial<=500)).nonzero()

    ar[a,b]=(np.floor(((trial[a,b]-(-500))/(500-(-500)))*100)+75)#at[a,b]


    #at=(np.floor(((trial-(500))/(1000-(500)))*25)+175)
    (a,b)=((500<trial) & (trial<=1000)).nonzero()
    ar[a,b]=(np.floor(((trial[a,b]-(500))/(1000-(500)))*25)+175)#at[a,b]



    #at=(np.floor(((trial-(1000))/(3000-(1000)))*25)+200)
    (a,b)=((1000<trial) & (trial<=3000)).nonzero()
    ar[a,b]=(np.floor(((trial[a,b]-(1000))/(3000-(1000)))*25)+200)#at[a,b]

    #at=(np.floor(((trial-(3000))/(np.max(trial)-(3000)))*25)+225)
    (a,b)=(3000<trial).nonzero()
    ar[a,b]=(np.floor(((trial[a,b]-(3000))/(np.max(trial)-(3000)))*25)+225)#at[a,b]
    
    #r=ar[:,:,0]
    #ab=np.ones(np.shape(r))
    
    return (ar)

In [62]:
from scipy.signal import hilbert
from scipy.stats import skew, entropy

These are for Bmode

In [ ]:
from skimage.transform import resize

In [ ]:
from scipy.ndimage.filters import uniform_filter, median_filter
from scipy.ndimage.measurements import variance
    ''' sanse: we-save B-modes, which potentially will be used in neural nets (new project)'''
def lee_filter(img, size):
    img_mean = uniform_filter(img, (size[0], size[1]))
    img_sqr_mean = uniform_filter(img**2, (size[0], size[1]))
    img_variance = img_sqr_mean - img_mean**2

    overall_variance = variance(img)

    img_weights = (img_variance / (img_variance + overall_variance))
    img_output = img_mean + img_weights * (img - img_mean)
    return img_output

In [ ]:
def scanconvert2(Iin=None, info=None):#Apitch, Lpitch, Radius, PixelsPerMM)
    ''' sanse: faster version of scan conversion, not deployed in MaskingCleaned (check on order of indices, it changes)'''
    ''' sanse: could be updated in MaskingCleaned (check)'''
    
    import numpy as np
    if (info["Radius"] == 0):    # A linear array probe
        print("Linear Array")
        AxialExtent = info["Apitch"] * np.size(Iin, 0)
        LateralExtent = info["Lpitch"] * np.size(Iin, 1)
        AxialSize = AxialExtent * 1000 * info["PixelsPerMM"]
        LateralSize = LateralExtent * 1000 * info["PixelsPerMM"]
        Iout = np.resize(Iin, (LateralSize, AxialSize))    # cubic interpolation
    else:    # A curved array probe    
        t = ((np.arange(1,Iin.shape[2],dtype=int)) - Iin.shape[2]/ 2) * info["Lpitch"] / info["Radius"]
        r = info["Radius"] + (np.arange(1,Iin.shape[1])) * info["Apitch"]
        [t, r] = np.meshgrid(t, r)
        x = np.multiply(r, np.cos(t))
        y = np.multiply(r, np.sin(t))
    
        divides=1e-3 / info["PixelsPerMM"]
        xMin=np.min(x)
        xMax=np.max(x)
        #print("X Min : %s , X Max %s" % (xMin,xMax))
        #print("Divides : %s " % divides)
        xreg =  np.arange(np.min(x) , np.max(x) ,divides)
        yreg =  np.arange(np.min(y) , np.max(y) ,divides)
        [yreg, xreg] = np.meshgrid(yreg, xreg) 
        temp=np.hstack((np.shape(Iin)[0],xreg.shape))
        Iout = np.zeros(temp)#xreg.shape)
        xCntr = np.arange(0,xreg.shape[1])
        Cntry = np.arange(0,yreg.shape[0])
        theta = np.array([cart4pol(xreg[yCntr, xCntr], yreg[yCntr, xCntr]) for yCntr in Cntry])
        rho = np.array([cart2pol(xreg[yCntr, xCntr], yreg[yCntr, xCntr]) for yCntr in Cntry])
        #for xCntr in tqdm(np.arange(0,xreg.shape[1])):
            #for yCntr in np.arange(0,yreg.shape[0]):
                #[theta, rho] = cart2pol(xreg[yCntr, xCntr], yreg[yCntr, xCntr])
                #print("theta : %s , rho %s" % (theta,rho))
        indt = np.floor(theta / (info["Lpitch"] / info["Radius"]) + Iin.shape[2] / 2) 
        indr = np.floor((rho - info["Radius"]) / info["Apitch"]) 

        [j,i]=((indt>=0)&(indt <= t.shape[1])& (indr>= 0) &(indr <= r.shape[0])).nonzero()
        #print(np.size(j))
        j=j.astype(int)
        i=i.astype(int)
        Iout[:,j, i] = Iin[:,indr[j,i].astype(int), indt[j,i].astype(int)]
        #for i in xCntr:
            
         #   for j in Cntry:
                
          #      if indt[j,i] >= 0 and (indt[j,i] <= t.shape[1]and (indr[j,i] >= 0 and indr[j,i] <= r.shape[0])):
                    
           #         Iout[int(Cntry[j]), int(xCntr[i])] = Iin[int(indr[j,i]), int(indt[j,i])]
    #print(np.shape(Iout))
    return Iout

def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    return rho
def cart4pol(x, y):
    phi = np.arctan2(y, x)
    return phi


def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)


In [ ]:
import matplotlib.pyplot as plt
from scipy.signal import hilbert
import cv2

### HEY ITS THIS ONE


In [ ]:
'''Add here new cell from adi'''

In [60]:
1+1

2

In [76]:
%%time

trainvidtopat_h0, trainvid_h0, trainlabvid_h0, trainframetovid_h0, trainframe_h0, trainwind_h0,trainlabframe_h0, trainweightrfh0,trainweightqnth0,windtoframeh0 = load_qus_textft(h3,h3k[0][5:8],0,h3kpa,h3cap,ep)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log
  if sys.path[0] == '':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in less
  app.launch_new_instance()



CPU times: user 29min 23s, sys: 52 s, total: 30min 15s
Wall time: 9min 56s


In [77]:
np.shape(np.hstack((trainweightqnth0)))

(413,)

In [78]:
np.sum(trainframetovid_h0)

413

In [51]:
%%time

trainvidtopat_h0, trainvid_h0, trainlabvid_h0, trainframetovid_h0, trainframe_h0, trainwind_h0,trainlabframe_h0, trainweightrfh0,trainweightqnth0,windtoframeh0 = load_qus_textft(h3,h3k[0][0:2],0,h3kpa,h3cap,ep)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log
  if sys.path[0] == '':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in less
  app.launch_new_instance()
/opt/conda/lib/python3.7/site-packages/hfda/core.py:67: RuntimeWarning: divide by zero encountered in log2
  D, _ = - np.polyfit(np.log2(k), np.log2(L), 1)
/opt/conda/lib/python3.7/site-packages/hfda/core.py:12: RuntimeWarning: invalid value encountered in long_scalars
  norm = (N-1) / (n*k)
/opt/conda/lib/python3.7/site-packages/hfda/core.py:12: RuntimeWarning: divide by zero encountered in long_scalars
  norm = (N-1) / (n*k)
/opt/conda/lib/python3.7/site-packages/hfda/core.py:16: RuntimeWarning: invalid value encountered in double_scalars
  Lm = (sum*norm) / k



CPU times: user 5min 49s, sys: 16.8 s, total: 6min 5s
Wall time: 6min 7s


In [55]:
merg=np.empty([204,476])
for i in range(0,204):
    merg[i,:]=trainframe_h0[i][-476:]

In [56]:
np.unique(merg[:,6])

array([ 9.33719479,  9.81720534, 10.26744977, 10.33130126, 10.57594835,
       10.82799624, 10.93369941, 11.19675351, 11.29763137, 11.3750424 ,
       11.45497232, 11.61073424, 11.7376362 , 11.79587503, 11.85424859,
       11.89540029, 11.90639763, 11.96253867, 11.99953923, 12.09351194,
       12.12113148, 12.13703822, 12.16762823, 12.22195895, 12.41446848,
       12.46786665, 12.66943639, 12.70196719, 12.71801627, 12.85731524,
       12.96462994, 12.9692167 , 12.98949348, 13.02753159, 13.12185371,
       13.26199741, 13.27635832, 13.35431483, 13.45333348, 13.46079853,
       13.46236106, 13.47566758, 13.55927461, 13.56912389, 13.60679363,
       13.63465513, 13.65780226, 13.67660495, 13.67802869, 13.68058463,
       13.70906076, 13.74706385, 13.7765663 , 13.81584354, 13.86549071,
       13.8755452 , 13.90704032, 13.91048792, 13.92149204, 13.92525028,
       13.92865011, 13.9774431 , 13.98266601, 14.0611594 , 14.08601579,
       14.08859298, 14.09805075, 14.10566954, 14.10817613, 14.11

In [ ]:
trainvidtopat_c0, trainvid_c0, trainlabvid_c0, trainframetovid_c0, trainframe_c0, trainwind_c0, trainlabframe_c0,trainweightrfc0,trainweightqntc0,windtoframec0 = load_qus_textft(c3,c3k,3,c3kpa,c3cap,ep)#3)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log
  if sys.path[0] == '':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in less
  app.launch_new_instance()
/opt/conda/lib/python3.7/site-packages/hfda/core.py:67: RuntimeWarning: divide by zero encountered in log2
  D, _ = - np.polyfit(np.log2(k), np.log2(L), 1)
/opt/conda/lib/python3.7/site-packages/hfda/core.py:12: RuntimeWarning: invalid value encountered in long_scalars
  norm = (N-1) / (n*k)
/opt/conda/lib/python3.7/site-packages/hfda/core.py:12: RuntimeWarning: divide by zero encountered in long_scalars
  norm = (N-1) / (n*k)
/opt/conda/lib/python3.7/site-packages/hfda/core.py:16: RuntimeWarning: invalid value encountered in double_scalars
  Lm = (sum*norm) / k


(17, 24)
(17, 4)
(408,)
(68,)


In [ ]:
trainvidtopat_h0, trainvid_h0, trainlabvid_h0, trainframetovid_h0, trainframe_h0, trainwind_h0,trainlabframe_h0, trainweightrfh0,trainweightqnth0,windtoframeh0 ,trainwinloch0,trainstatloc_h0, trainsimstats_h0, trainpowerspec_h0,trainBmodeh0   = load_qus_textft(h3,h3k[0],0,h3kpa,h3cap,ep)
trainvidtopat_f20, trainvid_f20, trainlabvid_f20, trainframetovid_f20, trainframe_f20, trainwind_f20, trainlabframe_f20,trainweightrff20,trainweightqntf20,windtoframef20,trainwinlocf20,trainstatloc_f20, trainsimstats_f20, trainpowerspec_f20,trainBmodef20= load_qus_textft(f23,f2k[0],1,f2kpa,f2cap,ep)
trainvidtopat_f30, trainvid_f30, trainlabvid_f30, trainframetovid_f30, trainframe_f30, trainwind_f30, trainlabframe_f30,trainweightrff30,trainweightqntf30,windtoframef30, trainwinlocf30,trainstatloc_f30, trainsimstats_f30, trainpowerspec_f30,trainBmodef30= load_qus_textft(f33,f3k[0],2,f3kpa,f3cap,ep)
trainvidtopat_c0, trainvid_c0, trainlabvid_c0, trainframetovid_c0, trainframe_c0, trainwind_c0, trainlabframe_c0,trainweightrfc0,trainweightqntc0,windtoframec0,trainwinlocc0,trainstatloc_c0, trainsimstats_c0, trainpowerspec_c0 ,trainBmodec0   = load_qus_textft(c3,c3k[0],3,c3kpa,c3cap,ep)#3)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log
  if sys.path[0] == '':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in less
  app.launch_new_instance()


In [ ]:
trainvidtopat_h1, trainvid_h1, trainlabvid_h1, trainframetovid_h1, trainframe_h1, trainwind_h1,trainlabframe_h1, trainweightrfh1,trainweightqnth1,windtoframeh1 ,trainwinloch1,trainstatloc_h1, trainsimstats_h1, trainpowerspec_h1  ,trainBmodeh1 = load_qus_textft(h3,h3k[1],0,h3kpa,h3cap,ep)
trainvidtopat_f21, trainvid_f21, trainlabvid_f21, trainframetovid_f21, trainframe_f21, trainwind_f21, trainlabframe_f21,trainweightrff21,trainweightqntf21,windtoframef21,trainwinlocf21,trainstatloc_f21, trainsimstats_f21, trainpowerspec_f21,trainBmodef21= load_qus_textft(f23,f2k[1],1,f2kpa,f2cap,ep)
trainvidtopat_f31, trainvid_f31, trainlabvid_f31, trainframetovid_f31, trainframe_f31, trainwind_f31, trainlabframe_f31,trainweightrff31,trainweightqntf31,windtoframef31,trainwinlocf31,trainstatloc_f31, trainsimstats_f31, trainpowerspec_f31,trainBmodef31 = load_qus_textft(f33,f3k[1],2,f3kpa,f3cap,ep)
trainvidtopat_c1, trainvid_c1, trainlabvid_c1, trainframetovid_c1, trainframe_c1, trainwind_c1, trainlabframe_c1,trainweightrfc1,trainweightqntc1,windtoframec1,trainwinlocc1,trainstatloc_c1, trainsimstats_c1, trainpowerspec_c1 ,trainBmodec1  = load_qus_textft(c3,c3k[1],3,c3kpa,c3cap,ep)#3

In [ ]:
trainvidtopat_h2, trainvid_h2, trainlabvid_h2, trainframetovid_h2, trainframe_h2, trainwind_h2,trainlabframe_h2, trainweightrfh2,trainweightqnth2,windtoframeh2,trainwinloch2 ,trainstatloc_h2, trainsimstats_h2, trainpowerspec_h2   ,trainBmodeh2 = load_qus_textft(h3,h3k[2],0,h3kpa,h3cap,ep)
trainvidtopat_f22, trainvid_f22, trainlabvid_f22, trainframetovid_f22, trainframe_f22, trainwind_f22, trainlabframe_f22,trainweightrff22,trainweightqntf22,windtoframef22,trainwinlocf22,trainstatloc_f22, trainsimstats_f22, trainpowerspec_f22,trainBmodef22= load_qus_textft(f23,f2k[2],1,f2kpa,f2cap,ep)
trainvidtopat_f32, trainvid_f32, trainlabvid_f32, trainframetovid_f32, trainframe_f32, trainwind_f32, trainlabframe_f32,trainweightrff32,trainweightqntf32,windtoframef32,trainwinlocf32,trainstatloc_f32, trainsimstats_f32, trainpowerspec_f32,trainBmodef32 = load_qus_textft(f33,f3k[2],2,f3kpa,f3cap,ep)
trainvidtopat_c2, trainvid_c2, trainlabvid_c2, trainframetovid_c2, trainframe_c2, trainwind_c2, trainlabframe_c2,trainweightrfc2,trainweightqntc2,windtoframec2,trainwinlocc2 ,trainstatloc_c2, trainsimstats_c2, trainpowerspec_c2 ,trainBmodec2  = load_qus_textft(c3,c3k[2],3,c3kpa,c3cap,ep)#3)

In [ ]:
trainvidtopat_h3, trainvid_h3, trainlabvid_h3, trainframetovid_h3, trainframe_h3, trainwind_h3,trainlabframe_h3, trainweightrfh3,trainweightqnth3 ,windtoframeh3,trainwinloch3,trainstatloc_h3, trainsimstats_h3, trainpowerspec_h3  trainBmodeh3 = load_qus_textft(h3,h3k[3],0,h3kpa,h3cap,ep)
trainvidtopat_f23, trainvid_f23, trainlabvid_f23, trainframetovid_f23, trainframe_f23, trainwind_f23, trainlabframe_f23,trainweightrff23,trainweightqntf23,windtoframef23,trainwinlocf23,trainstatloc_f23, trainsimstats_f23, trainpowerspec_f23trainBmodef23= load_qus_textft(f23,f2k[3],1,f2kpa,f2cap,ep)
trainvidtopat_f33, trainvid_f33, trainlabvid_f33, trainframetovid_f33, trainframe_f33, trainwind_f33, trainlabframe_f33,trainweightrff33,trainweightqntf33,windtoframef33,trainwinlocf33,trainstatloc_f33, trainsimstats_f33, trainpowerspec_f33trainBmodef33 = load_qus_textft(f33,f3k[3],2,f3kpa,f3cap,ep)
trainvidtopat_c3, trainvid_c3, trainlabvid_c3, trainframetovid_c3, trainframe_c3, trainwind_c3, trainlabframe_c3,trainweightrfc3,trainweightqntc3,windtoframec3,trainwinlocc3  ,trainstatloc_c3, trainsimstats_c3, trainpowerspec_c3 trainBmodec3  = load_qus_textft(c3,c3k[3],3,c3kpa,c3cap,ep)#3)

In [ ]:
trainvidtopat_h4, trainvid_h4, trainlabvid_h4, trainframetovid_h4, trainframe_h4, trainwind_h4,trainlabframe_h4, trainweightrfh4,trainweightqnth4 ,windtoframeh4,trainwinloch4,trainstatloc_h4, trainsimstats_h4, trainpowerspec_h4 , trainBmodeh4 = load_qus_textft(h3,h3k[4],0,h3kpa,h3cap,ep)
trainvidtopat_f24, trainvid_f24, trainlabvid_f24, trainframetovid_f24, trainframe_f24, trainwind_f24, trainlabframe_f24,trainweightrff24,trainweightqntf24,windtoframef24,trainwinlocf24,trainstatloc_f24, trainsimstats_f24, trainpowerspec_f24,trainBmodef24= load_qus_textft(f23,f2k[4],1,f2kpa,f2cap,ep)
trainvidtopat_f34, trainvid_f34, trainlabvid_f34, trainframetovid_f34, trainframe_f34, trainwind_f34, trainlabframe_f34,trainweightrff34,trainweightqntf34,windtoframef34,trainwinlocf34,trainstatloc_f34, trainsimstats_f34, trainpowerspec_f34,trainBmodef34 = load_qus_textft(f33,f3k[4],2,f3kpa,f3cap,ep)
trainvidtopat_c4, trainvid_c4, trainlabvid_c4, trainframetovid_c4, trainframe_c4, trainwind_c4, trainlabframe_c4,trainweightrfc4,trainweightqntc4,windtoframec4,trainwinlocc4,trainstatloc_c4, trainsimstats_c4, trainpowerspec_c4 , trainBmodec4 = load_qus_textft(c3,c3k[4],3,c3kpa,c3cap,ep)#3)

In [75]:
trainvidtopat_c3

[6, 3, 4, 7, 4, 6, 1, 8, 7, 4, 4, 3, 8, 4]

In [ ]:
##run this one

trainvidtopat1 = np.concatenate((trainvidtopat_h1,trainvidtopat_f21,trainvidtopat_f31,trainvidtopat_c1))#, trainvidtopat_f41))
trainvid1 = np.concatenate((trainvid_h1, trainvid_f21,trainvid_f31, trainvid_c1))#, trainvid_f41))
trainlabvid1 = np.concatenate((trainlabvid_h1, trainlabvid_f21,trainlabvid_f31, trainlabvid_c1))#, trainlabvid_f41))
trainframetovid1 = np.concatenate((trainframetovid_h1,trainframetovid_f21,trainframetovid_f31,trainframetovid_c1))#,trainframetovid_f41))
trainframe1 = np.concatenate((trainframe_h1,trainframe_f21,trainframe_f31,trainframe_c1))#,trainframe_f41))
trainwind1 = np.concatenate((trainwind_h1,trainwind_f21,trainwind_f31,trainwind_c1))#,trainwind_f41))
trainlabframe1 = np.concatenate((trainlabframe_h1,trainlabframe_f21,trainlabframe_f31,trainlabframe_c1))#,trainlabframe_f41))

trainvidtopat2 = np.concatenate((trainvidtopat_h2,trainvidtopat_f22,trainvidtopat_f32,trainvidtopat_c2))#, trainvidtopat_f42))
trainvid2 = np.concatenate((trainvid_h2, trainvid_f22,trainvid_f32, trainvid_c2))#, trainvid_f42))
trainlabvid2 = np.concatenate((trainlabvid_h2,  trainlabvid_f22,trainlabvid_f32, trainlabvid_c2))#, trainlabvid_f42))
trainframetovid2 = np.concatenate((trainframetovid_h2,trainframetovid_f22,trainframetovid_f32,trainframetovid_c2))#,trainframetovid_f42))
trainframe2 = np.concatenate((trainframe_h2,trainframe_f22,trainframe_f32,trainframe_c2))#,trainframe_f42))
trainwind2 = np.concatenate((trainwind_h2,trainwind_f22,trainwind_f32,trainwind_c2))#,trainwind_f42))
trainlabframe2 = np.concatenate((trainlabframe_h2,trainlabframe_f22,trainlabframe_f32,trainlabframe_c2))#,trainlabframe_f42))




trainvidtopat0 = np.concatenate((trainvidtopat_h0,trainvidtopat_f20,trainvidtopat_f30,trainvidtopat_c0))#, trainvidtopat_f41))
trainvid0 = np.concatenate((trainvid_h0,trainvid_f20,trainvid_f30, trainvid_c0))#, trainvid_f41))
trainlabvid0 = np.concatenate((trainlabvid_h0,  trainlabvid_f20,trainlabvid_f30, trainlabvid_c0))#, trainlabvid_f41))
trainframetovid0 = np.concatenate((trainframetovid_h0,trainframetovid_f20,trainframetovid_f30,trainframetovid_c0))#,trainframetovid_f41))
trainframe0 = np.concatenate((trainframe_h0,trainframe_f20,trainframe_f30,trainframe_c0))#,trainframe_f41))
trainwind0 = np.concatenate((trainwind_h0,trainwind_f20,trainwind_f30,trainwind_c0))#,trainwind_f41))
trainlabframe0 = np.concatenate((trainlabframe_h0,trainlabframe_f20,trainlabframe_f30,trainlabframe_c0))#,trainlabframe_f41))


trainvidtopat3 = np.concatenate((trainvidtopat_h3,trainvidtopat_f23,trainvidtopat_f33,trainvidtopat_c3))#, trainvidtopat_f43))
trainvid3 = np.concatenate((trainvid_h3, trainvid_f23,trainvid_f33, trainvid_c3))#, trainvid_f43))
trainlabvid3 = np.concatenate((trainlabvid_h3, trainlabvid_f23,trainlabvid_f33, trainlabvid_c3))#, trainlabvid_f43))
trainframetovid3 = np.concatenate((trainframetovid_h3,trainframetovid_f23,trainframetovid_f33,trainframetovid_c3))#,trainframetovid_f43))
trainframe3 = np.concatenate((trainframe_h3,trainframe_f23,trainframe_f33,trainframe_c3))#,trainframe_f43))
trainwind3 = np.concatenate((trainwind_h3,trainwind_f23,trainwind_f33,trainwind_c3))#,trainwind_f43))
trainlabframe3 = np.concatenate((trainlabframe_h3,trainlabframe_f23,trainlabframe_f33,trainlabframe_c3))#,trainlabframe_f43))



trainvidtopat4 = np.concatenate((trainvidtopat_h4,trainvidtopat_f24,trainvidtopat_f34,trainvidtopat_c4))#, trainvidtopat_f41))
trainvid4 = np.concatenate((trainvid_h4, trainvid_f24,trainvid_f34, trainvid_c4))#, trainvid_f41))
trainlabvid4 = np.concatenate((trainlabvid_h4,  trainlabvid_f24,trainlabvid_f34, trainlabvid_c4))#, trainlabvid_f41))
trainframetovid4 = np.concatenate((trainframetovid_h4,trainframetovid_f24,trainframetovid_f34,trainframetovid_c4))#,trainframetovid_f41))
trainframe4 = np.concatenate((trainframe_h4,trainframe_f24,trainframe_f34,trainframe_c4))#,trainframe_f41))
trainwind4 = np.concatenate((trainwind_h4,trainwind_f24,trainwind_f34,trainwind_c4))#,trainwind_f41))
trainlabframe4 = np.concatenate((trainlabframe_h4,trainlabframe_f24,trainlabframe_f34,trainlabframe_c4))#,trainlabframe_f41))


In [ ]:

#run this one
trainweightrf1 = np.concatenate((trainweightrfh1,trainweightrff21,trainweightrff31,trainweightrfc1))#,trainweightrff41))
trainweightrf2 = np.concatenate((trainweightrfh2,trainweightrff22,trainweightrff32,trainweightrfc2))#,trainweightrff42))
trainweightrf3 = np.concatenate((trainweightrfh3,trainweightrff23,trainweightrff33,trainweightrfc3))#,trainweightrff43))
trainweightrf0 = np.concatenate((trainweightrfh0,trainweightrff20,trainweightrff30,trainweightrfc0))#,trainweightrff41))
trainweightrf4 = np.concatenate((trainweightrfh4,trainweightrff24,trainweightrff34,trainweightrfc4))#,trainweightrff41))


trainweightqnt1 = np.concatenate((trainweightqnth1,trainweightqntf21,trainweightqntf31,trainweightqntc1))#,trainweightqntf41))
trainweightqnt2 = np.concatenate((trainweightqnth2,trainweightqntf22,trainweightqntf32,trainweightqntc2))#,trainweightqntf42))
trainweightqnt3 = np.concatenate((trainweightqnth3,trainweightqntf23,trainweightqntf33,trainweightqntc3))#,trainweightqntf43))
trainweightqnt0 = np.concatenate((trainweightqnth0,trainweightqntf20,trainweightqntf30,trainweightqntc0))#,trainweightqntf41))
trainweightqnt4 = np.concatenate((trainweightqnth4,trainweightqntf24,trainweightqntf34,trainweightqntc4))#,trainweightqntf41))

windtoframe1 = np.concatenate((windtoframeh1,windtoframef21,windtoframef31,windtoframec1))#,trainweightqntf41))
windtoframe2 = np.concatenate((windtoframeh2,windtoframef22,windtoframef32,windtoframec2))#,trainweightqntf42))
windtoframe3 = np.concatenate((windtoframeh3,windtoframef23,windtoframef33,windtoframec3))#,trainweightqntf43))
windtoframe0 = np.concatenate((windtoframeh0,windtoframef20,windtoframef30,windtoframec0))#,trainweightqntf41))
windtoframe4 = np.concatenate((windtoframeh4,windtoframef24,windtoframef34,windtoframec4))#,trainweightqntf41))

trainwinloc1 = np.concatenate((trainwinloch1,trainwinlocf21,trainwinlocf31,trainwinlocc1))#,trainweightqntf41))
trainwinloc2 = np.concatenate((trainwinloch2,trainwinlocf22,trainwinlocf32,trainwinlocc2))#,trainweightqntf42))
trainwinloc3 = np.concatenate((trainwinloch3,trainwinlocf23,trainwinlocf33,trainwinlocc3))#,trainweightqntf43))
trainwinloc0 = np.concatenate((trainwinloch0,trainwinlocf20,trainwinlocf30,trainwinlocc0))#,trainweightqntf41))
trainwinloc4 = np.concatenate((trainwinloch4,trainwinlocf24,trainwinlocf34,trainwinlocc4))#,trainweightqntf41))

trainpowerspec1 = np.concatenate((trainpowerspec_h1,trainpowerspec_f21,trainpowerspec_f31,trainpowerspec_c1))#,trainweightrff41))
trainpowerspec2 = np.concatenate((trainpowerspec_h2,trainpowerspec_f22,trainpowerspec_f32,trainpowerspec_c2))#,trainweightrff42))
trainpowerspec3 = np.concatenate((trainpowerspec_h3,trainpowerspec_f23,trainpowerspec_f33,trainpowerspec_c3))#,trainweightrff43))
trainpowerspec0 = np.concatenate((trainpowerspec_h0,trainpowerspec_f20,trainpowerspec_f30,trainpowerspec_c0))#,trainweightrff41))
trainpowerspec4 = np.concatenate((trainpowerspec_h4,trainpowerspec_f24,trainpowerspec_f34,trainpowerspec_c4))#,trainweightrff41))


trainsimstats1 = np.concatenate((trainsimstats_h1,trainsimstats_f21,trainsimstats_f31,trainsimstats_c1))#,trainweightqntf41))
trainsimstats2 = np.concatenate((trainsimstats_h2,trainsimstats_f22,trainsimstats_f32,trainsimstats_c2))#,trainweightqntf42))
trainsimstats3 = np.concatenate((trainsimstats_h3,trainsimstats_f23,trainsimstats_f33,trainsimstats_c3))#,trainweightqntf43))
trainsimstats0 = np.concatenate((trainsimstats_h0,trainsimstats_f20,trainsimstats_f30,trainsimstats_c0))#,trainweightqntf41))
trainsimstats4 = np.concatenate((trainsimstats_h4,trainsimstats_f24,trainsimstats_f34,trainsimstats_c4))#,trainweightqntf41))

trainstatloc1 = np.concatenate((trainstatloc_h1,trainstatloc_f21,trainstatloc_f31,trainstatloc_c1))#,trainweightqntf41))
trainstatloc2 = np.concatenate((trainstatloc_h2,trainstatloc_f22,trainstatloc_f32,trainstatloc_c2))#,trainweightqntf42))
trainstatloc3 = np.concatenate((trainstatloc_h3,trainstatloc_f23,trainstatloc_f33,trainstatloc_c3))#,trainweightqntf43))
trainstatloc0 = np.concatenate((trainstatloc_h0,trainstatloc_f20,trainstatloc_f30,trainstatloc_c0))#,trainweightqntf41))
trainstatloc4 = np.concatenate((trainstatloc_h4,trainstatloc_f24,trainstatloc_f34,trainstatloc_c4))#,trainweightqntf41))

trainBmode1 = np.concatenate((trainBmodeh1,trainBmodef21,trainBmodef31,trainBmodec1))#,trainweightqntf41))
trainBmode2 = np.concatenate((trainBmodeh2,trainBmodef22,trainBmodef32,trainBmodec2))#,trainweightqntf42))
trainBmode3 = np.concatenate((trainBmodeh3,trainBmodef23,trainBmodef33,trainBmodec3))#,trainweightqntf43))
trainBmode0 = np.concatenate((trainBmodeh0,trainBmodef20,trainBmodef30,trainBmodec0))#,trainweightqntf41))
trainBmode4 = np.concatenate((trainBmodeh4,trainBmodef24,trainBmodef34,trainBmodec4))#,trainweightqntf41))


In [1]:
#run this one
np.save('/home/jupyter/AdisStuff/kfold_new38/trainwinloc0_kfold' ,trainwinloc0)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainwinloc2_kfold' ,trainwinloc2)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainwinloc3_kfold' ,trainwinloc3)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainwinloc1_kfold' ,trainwinloc1)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainwinloc4_kfold' ,trainwinloc4)

np.save('/home/jupyter/AdisStuff/kfold_new38/trainlabframe0_kfold' ,trainlabframe0)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainframe0_kfold' ,trainframe0)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainwind0_kfold' ,trainwind0)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainlabvid0_kfold' ,trainlabvid0)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainvid0_kfold' ,trainvid0)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainframetovid0_kfold' ,trainframetovid0)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainvidtopat0_kfold' ,trainvidtopat0)


np.save('/home/jupyter/AdisStuff/kfold_new38/trainlabframe1_kfold' ,trainlabframe1)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainframe1_kfold' ,trainframe1)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainwind1_kfold' ,trainwind1)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainlabvid1_kfold' ,trainlabvid1)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainvid1_kfold' ,trainvid1)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainframetovid1_kfold' ,trainframetovid1)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainvidtopat1_kfold' ,trainvidtopat1)


np.save('/home/jupyter/AdisStuff/kfold_new38/trainlabframe2_kfold' ,trainlabframe2)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainframe2_kfold' ,trainframe2)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainwind2_kfold' ,trainwind2)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainlabvid2_kfold' ,trainlabvid2)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainvid2_kfold' ,trainvid2)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainframetovid2_kfold' ,trainframetovid2)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainvidtopat2_kfold' ,trainvidtopat2)

np.save('/home/jupyter/AdisStuff/kfold_new38/trainlabframe3_kfold' ,trainlabframe3)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainframe3_kfold' ,trainframe3)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainwind3_kfold' ,trainwind3)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainlabvid3_kfold' ,trainlabvid3)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainvid3_kfold' ,trainvid3)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainframetovid3_kfold' ,trainframetovid3)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainvidtopat3_kfold' ,trainvidtopat3)


np.save('/home/jupyter/AdisStuff/kfold_new38/trainlabframe4_kfold' ,trainlabframe4)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainframe4_kfold' ,trainframe4)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainwind4_kfold' ,trainwind4)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainlabvid4_kfold' ,trainlabvid4)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainvid4_kfold' ,trainvid4)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainframetovid4_kfold' ,trainframetovid4)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainvidtopat4_kfold' ,trainvidtopat4)

np.save('/home/jupyter/AdisStuff/kfold_new38/trainweightrf0_kfold' ,trainweightrf0)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainweightrf2_kfold' ,trainweightrf2)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainweightrf3_kfold' ,trainweightrf3)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainweightrf1_kfold' ,trainweightrf1)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainweightrf4_kfold' ,trainweightrf4)



np.save('/home/jupyter/AdisStuff/kfold_new38/trainweightqnt0_kfold' ,trainweightqnt0)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainweightqnt2_kfold' ,trainweightqnt2)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainweightqnt3_kfold' ,trainweightqnt3)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainweightqnt1_kfold' ,trainweightqnt1)
np.save('/home/jupyter/AdisStuff/kfold_new38/trainweightqnt4_kfold' ,trainweightqnt4)

np.save('/home/jupyter/AdisStuff/kfold_new38/windtoframe0_kfold' ,windtoframe0)
np.save('/home/jupyter/AdisStuff/kfold_new38/windtoframe2_kfold' ,windtoframe2)
np.save('/home/jupyter/AdisStuff/kfold_new38/windtoframe3_kfold' ,windtoframe3)
np.save('/home/jupyter/AdisStuff/kfold_new38/windtoframe1_kfold' ,windtoframe1)
np.save('/home/jupyter/AdisStuff/kfold_new38/windtoframe4_kfold' ,windtoframe4)


np.save('/home/jupyter/AdisStuff/kfold_new38_L/trainpowerspec0_kfold' ,trainpowerspec0)
np.save('/home/jupyter/AdisStuff/kfold_new38_L/trainpowerspec2_kfold' ,trainpowerspec2)
np.save('/home/jupyter/AdisStuff/kfold_new38_L/trainpowerspec3_kfold' ,trainpowerspec3)
np.save('/home/jupyter/AdisStuff/kfold_new38_L/trainpowerspec1_kfold' ,trainpowerspec1)
np.save('/home/jupyter/AdisStuff/kfold_new38_L/trainpowerspec4_kfold' ,trainpowerspec4)



np.save('/home/jupyter/AdisStuff/kfold_new38_L/trainsimstats0_kfold' ,trainsimstats0)
np.save('/home/jupyter/AdisStuff/kfold_new38_L/trainsimstats2_kfold' ,trainsimstats2)
np.save('/home/jupyter/AdisStuff/kfold_new38_L/trainsimstats3_kfold' ,trainsimstats3)
np.save('/home/jupyter/AdisStuff/kfold_new38_L/trainsimstats1_kfold' ,trainsimstats1)
np.save('/home/jupyter/AdisStuff/kfold_new38_L/trainsimstats4_kfold' ,trainsimstats4)

np.save('/home/jupyter/AdisStuff/kfold_new38_L/trainstatloc0_kfold' ,trainstatloc0)
np.save('/home/jupyter/AdisStuff/kfold_new38_L/trainstatloc2_kfold' ,trainstatloc2)
np.save('/home/jupyter/AdisStuff/kfold_new38_L/trainstatloc3_kfold' ,trainstatloc3)
np.save('/home/jupyter/AdisStuff/kfold_new38_L/trainstatloc1_kfold' ,trainstatloc1)
np.save('/home/jupyter/AdisStuff/kfold_new38_L/trainstatloc4_kfold' ,trainstatloc4)


np.save('/home/jupyter/AdisStuff/kfold_new38_L/trainBmode0_kfold' ,trainBmode0)
np.save('/home/jupyter/AdisStuff/kfold_new38_L/trainBmode2_kfold' ,trainBmode2)
np.save('/home/jupyter/AdisStuff/kfold_new38_L/trainBmode3_kfold' ,trainBmode3)
np.save('/home/jupyter/AdisStuff/kfold_new38_L/trainBmode1_kfold' ,trainBmode1)
np.save('/home/jupyter/AdisStuff/kfold_new38_L/trainBmode4_kfold' ,trainBmode4)


NameError: name 'np' is not defined

In [65]:
np.save('/home/jupyter/AdisStuff/kfold_new38/c3k' ,c3k)
np.save('/home/jupyter/AdisStuff/kfold_new38/f3k' ,f3k)
np.save('/home/jupyter/AdisStuff/kfold_new38/f2k' ,f2k)
np.save('/home/jupyter/AdisStuff/kfold_new38/h3k' ,h3k)

In [66]:
np.save('/home/jupyter/AdisStuff/kfold_new38/c3' ,c3)
np.save('/home/jupyter/AdisStuff/kfold_new38/f33' ,f33)
np.save('/home/jupyter/AdisStuff/kfold_new38/f23' ,f23)
np.save('/home/jupyter/AdisStuff/kfold_new38/h3' ,h3)

In [126]:
np.shape(c3k[0])

(15,)

In [ ]:
Qmap=np.zeros([1152,17])#
Qmap[(Qmap==0).nonzero()]=np.nan
Qmap[trainwinloc0[0],:]=trainwind0[0]
Qmap=np.reshape(Qmap,(24,48,17))